### Marcus Analysis

*Neal Xie*  
*2019-07-10*

#### - Package -

In [37]:
# Standard data science helpers
import numpy as np
import pandas as pd
import scipy

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# System display
import os
from IPython.display import Image, display, HTML

# Widgets control
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

#### - Data -

In [6]:
# DCM data
df_dcm = pd.read_excel('DCM.xlsx')
df_dcm = df_dcm.groupby(['Date', 'Month', 'Campaign', 'Site (DCM)', 'Package/Roadblock', 'Pod', 'Market', 'Site', 'Funnel', 'Week', 'Trigger', 'Creative Cleaned', 'DV360 eCPM Revised', 'YouTube eCPM'])[['Impressions', 'Clicks', 'Media Cost', 'Total Conversions', 'Aquisition : Marcus_Universal_Floodlight: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-AccountDecline_Hard: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-Step2_AppStart: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-OTP: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-Password/SecretWord: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-KBA: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-Step4_FundAccountGateway: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-FundAccountConfirmation: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-ESign: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-OnClick-DisclosureSubmitButton: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-AccountDecline_Soft: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-Dashboard: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-InstantVerification: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-InstantVerification_Amount: Total Conversions', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-Login: Total Conversions', 'P Video Views/Engagements', 'P Video Costs', 'YouTube Costs', 'Final Media Cost']].sum().reset_index()
df_dcm_week = df_dcm.groupby(['Week', 'Site', 'Pod', 'Market'])[['Impressions', 'Clicks', 'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-Step2_AppStart: Total Conversions', 'Aquisition : Marcus_Universal_Floodlight: Total Conversions', 'Media Cost', 'Final Media Cost']].sum().reset_index()

In [7]:
df_dcm_week.rename(columns = {'Banking : Marcus-US-Deposits-Marketing-CM-PageLoad-Step2_AppStart: Total Conversions':'AP1 Loads'}, inplace=True)
df_dcm_week.rename(columns = {'Aquisition : Marcus_Universal_Floodlight: Total Conversions':'Landing Page Events'}, inplace=True)

In [8]:
df_dcm_week = df_dcm_week.groupby(['Week', 'Pod', 'Market'])[['Impressions', 'Clicks', 'AP1 Loads', 'Landing Page Events', 'Final Media Cost']].sum().reset_index()

In [9]:
df_dcm_week = df_dcm_week[df_dcm_week['Market'].isin(['Chicago', 'Washington, DC', 'Denver', 'Hartford', 'Minneapolis', 'San Diego', 'Philadelphia', 'San Francisco'])]

In [10]:
df_dcm_week.loc[df_dcm_week['Market'] == 'Washington, DC', 'Market'] = 'Washington DC'
df_dcm_week.loc[df_dcm_week['Market'] == 'Hartford', 'Market'] = 'Hartford-New Haven'

In [11]:
df_dcm_week = df_dcm_week[(df_dcm_week['Week'] >= pd.Timestamp(2019,4,1)) & (df_dcm_week['Week'] <= pd.Timestamp(2019,7,1))]

In [12]:
df_dcm_week.head()

,Week,Pod,Market,Impressions,Clicks,AP1 Loads,Landing Page Events,Final Media Cost
10,2019-04-01,Pod 1,Chicago,491544.0,289,1,125,5407.810050
11,2019-04-01,Pod 1,Washington DC,232321.0,244,1,93,2058.862568
12,2019-04-01,Pod 2,Denver,528968.0,429,1,207,5283.425025
13,2019-04-01,Pod 2,Hartford-New Haven,439981.0,403,0,152,4062.548711
14,2019-04-01,Pod 2,Minneapolis,617997.0,546,1,214,5555.680626


In [13]:
# Account open data
df_ao = pd.read_excel('AccountOpen.xlsx')
df_ao_week = df_ao.set_index('vis_date').groupby(['Pod','Market','channel_id'])["e119_count"].resample("W-MON").sum().reset_index()
df_ao_week = df_ao_week[df_ao_week['channel_id'] == 'Direct']
df_ao_week.loc[df_ao_week['Pod'] == 1, 'Pod'] = 'Pod 1'
df_ao_week.loc[df_ao_week['Pod'] == 2, 'Pod'] = 'Pod 2'
df_ao_week.loc[df_ao_week['Pod'] == 3, 'Pod'] = 'Pod 3'
df_ao_week['Market'] = df_ao_week['Market'].str.strip()
df_ao_week = df_ao_week[df_ao_week['Market'].isin(['Chicago', 'Washington DC', 'Denver', 'Hartford-New Haven', 'Minneapolis', 'San Diego', 'Philadelphia', 'San Francisco'])]

In [14]:
df_ao_week.rename(columns = {'vis_date':'Week'}, inplace=True)
df_ao_week.rename(columns = {'e119_count':'Account Opens'}, inplace=True)

In [15]:
df_ao_week = df_ao_week[['Week', 'Pod', 'Market', 'Account Opens']]

In [16]:
df_ao_week.head()

,Week,Pod,Market,Account Opens
705,2018-01-01,Pod 1,Chicago,6
706,2018-01-08,Pod 1,Chicago,77
707,2018-01-15,Pod 1,Chicago,50
708,2018-01-22,Pod 1,Chicago,78
709,2018-01-29,Pod 1,Chicago,60


In [89]:
# Fianl dataset
df_final = pd.merge(df_dcm_week, df_ao_week, how='left', left_on=['Week', 'Pod', 'Market'], right_on=['Week', 'Pod', 'Market'])

In [93]:
df_final.head(15)

,Week,Pod,Market,Impressions,Clicks,AP1 Loads,Landing Page Events,Final Media Cost,Account Opens
0,2019-04-01,Pod 1,Chicago,491544.0,289,1,125,5407.810050,140
1,2019-04-01,Pod 1,Washington DC,232321.0,244,1,93,2058.862568,269
2,2019-04-01,Pod 2,Denver,528968.0,429,1,207,5283.425025,61
3,2019-04-01,Pod 2,Hartford-New Haven,439981.0,403,0,152,4062.548711,32
4,2019-04-01,Pod 2,Minneapolis,617997.0,546,1,214,5555.680626,35
5,2019-04-01,Pod 2,San Diego,367082.0,208,0,96,3909.668079,39
6,2019-04-01,Pod 3,Philadelphia,680310.0,563,1,238,6536.923212,105
7,2019-04-01,Pod 3,San Francisco,527390.0,505,2,214,4719.742815,248
8,2019-04-08,Pod 1,Chicago,622738.0,864,0,296,9021.584153,158
9,2019-04-08,Pod 1,Washington DC,318279.0,503,0,188,4363.858918,175


In [91]:
df_final.describe()

,Impressions,Clicks,AP1 Loads,Landing Page Events,Final Media Cost,Account Opens
count,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000
mean,481955.883929,738.866071,3.723214,408.946429,5241.274322,97.848214
std,175353.747533,288.700802,3.005130,152.965334,2830.819436,74.398877
min,126197.000000,208.000000,0.000000,93.000000,577.006957,11.000000
25%,347150.750000,530.750000,2.000000,302.500000,2923.372154,41.000000
50%,484519.000000,706.500000,3.000000,417.000000,4724.977730,66.000000
75%,614119.500000,934.750000,5.000000,512.750000,7390.513767,139.000000
max,968441.000000,1553.000000,18.000000,807.000000,12754.074043,338.000000


#### - Analysis -

In [92]:
# Correlation
@interact
def correlations(market=list(df_final.Market.unique()),
                 column1=list(df_final.select_dtypes('number').columns),
                 column2=list(df_final.select_dtypes('number').columns)):
    plot_df = df_final.loc[df_final.Market == market].copy()
    print(f"Correlation: {plot_df[column1].corr(plot_df[column2])}")

interactive(children=(Dropdown(description='market', options=('Chicago', 'Washington DC', 'Denver', 'Hartford-…

In [21]:
# Trend Performance
def plot_up_to(market=list(df_final.Market.unique()),
               KPI=list(df_final.select_dtypes('number').columns)):
    plot_df = df_final.loc[df_final.Market == market][['Week',KPI]].copy()
    plot_df.iplot(x='Week',
                  y=KPI,
                  mode='markers+lines',
                  xTitle='Week',
                  yTitle='KPIs',
                  title=f'Trend Performance')
_ = interact(plot_up_to)

interactive(children=(Dropdown(description='market', options=('Chicago', 'Washington DC', 'Denver', 'Hartford-…

In [23]:
# Forecasting model
import tensorflow as tf

In [24]:
df_final.head()

,Week,Pod,Market,Impressions,Clicks,AP1 Loads,Landing Page Events,Final Media Cost,Account Opens
0,2019-04-01,Pod 1,Chicago,491544.0,289,1,125,5407.810050,140
1,2019-04-01,Pod 1,Washington DC,232321.0,244,1,93,2058.862568,269
2,2019-04-01,Pod 2,Denver,528968.0,429,1,207,5283.425025,61
3,2019-04-01,Pod 2,Hartford-New Haven,439981.0,403,0,152,4062.548711,32
4,2019-04-01,Pod 2,Minneapolis,617997.0,546,1,214,5555.680626,35


In [76]:
df_final.rename(columns = {'AP1 Loads':'AP1_Loads'}, inplace=True)
df_final.rename(columns = {'Landing Page Events':'Landing_Page_Events'}, inplace=True)
df_final.rename(columns = {'Final Media Cost':'Final_Media_Cost'}, inplace=True)
df_final.rename(columns = {'Account Opens':'Account_Opens'}, inplace=True)

In [26]:
def get_feature_cols(features_df):
    feature_cols=[]
    for feature in list(features_df):
        feature_cols.append(tf.feature_column.numeric_column(str(feature)))
    return feature_cols

def get_input_fn(features_df, targets_df, num_epochs, batch_size=1, shuffle=True):
    input_fn = tf.estimator.inputs.pandas_input_fn(
        x=features_df,
        y=targets_df,
        num_epochs=num_epochs,
        batch_size=batch_size,
        shuffle=shuffle)
    return input_fn

In [27]:
input_df = df_final

train_df = input_df.sample(frac=0.8)
eval_df = input_df.drop(train_df.index)

train_features_df = train_df.drop(['Week', 'Pod', 'Market', 'Account_Opens'], axis=1)
train_targets_df = train_df['Account_Opens']
print('Training Samples {}'.format(len(train_df)))

eval_features_df = eval_df.drop(['Week', 'Pod', 'Market', 'Account_Opens'], axis=1)
eval_targets_df = eval_df['Account_Opens']
print('Evaluation Samples: {}'.format(len(eval_df)))

feature_cols = get_feature_cols(train_features_df)

Training Samples 90
Evaluation Samples: 22


In [28]:
train_input_fn = get_input_fn(
    features_df = train_features_df,
    targets_df = train_targets_df,
    num_epochs = 10,
    batch_size = 100,
    shuffle=True)

eval_input_fn = tf.estimator.inputs.pandas_input_fn(
    x = eval_features_df,
    y = eval_targets_df,
    shuffle=False)

In [29]:
lr = tf.estimator.LinearRegressor(feature_columns=feature_cols)
lr = lr.train(input_fn=train_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\yxie3\\AppData\\Local\\Temp\\tmpgiurlsfg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018748DB46A0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To 

In [30]:
metrics = lr.evaluate(input_fn=eval_input_fn)
print(metrics)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-07-11T14:35:38Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\yxie3\AppData\Local\Temp\tmpgiurlsfg\model.ckpt-9
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-07-11-14:35:38
INFO:tensorflow:Saving dict for global step 9: average_loss = 7827.5015, global_step = 9, label/mean = 98.86364, loss = 172205.03, prediction/mean = 79.66566
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 9: C:\Users\yxie3\AppData\Local\Temp\tmpgiurlsfg\model.ckpt-9
{'average_loss': 7827.5015, 'label/mean': 98.86364, 'loss': 172205.03, 'prediction/mean': 79.66566, 'global_step': 9}


In [31]:
num_features = len(train_df.columns)
weights = np.identity(num_features)

feature_names = []
for each in feature_cols:
    feature_names.append(each.key)
    
weights_dict = dict(zip(feature_names, weights))
weights_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=weights_dict,
    y=None,
    shuffle=False)

predictions = lr.predict(input_fn=weights_input_fn)
all_preds = [pred['predictions'][0] for pred in predictions]
feature_weights = zip(feature_names, all_preds)

sorted_feature_weights = sorted(feature_weights, key=lambda x: x[1], reverse=True)

print('Weights:', sorted_feature_weights)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\yxie3\AppData\Local\Temp\tmpgiurlsfg\model.ckpt-9
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Weights: [('AP1_Loads', 0.0007399779), ('Landing_Page_Events', 0.000648754), ('Clicks', 0.0006083973), ('Impressions', 0.00055803783), ('Final_Media_Cost', 0.0005568417)]


In [40]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

1.13.1


In [77]:
df_final['Market'].unique()

array(['Chicago', 'Washington DC', 'Denver', 'Hartford-New Haven',
       'Minneapolis', 'San Diego', 'Philadelphia', 'San Francisco'],
      dtype=object)

In [78]:
df_final = df_final.drop(['Week', 'Pod'], axis=1)

In [79]:
market = df_final.pop('Market')

In [80]:
df_final['Chicago'] = (market == 'Chicago')*1.0
df_final['Washington DC'] = (market == 'Washington DC')*1.0
df_final['Denver'] = (market == 'Denver')*1.0
df_final['Hartford-New Haven'] = (market == 'Hartford-New Haven')*1.0
df_final['Minneapolis'] = (market == 'Minneapolis')*1.0
df_final['San Diego'] = (market == 'San Diego')*1.0
df_final['Philadelphia'] = (market == 'Philadelphia')*1.0
df_final['San Francisco'] = (market == 'San Francisco')*1.0
df_final.tail()

,Impressions,Clicks,AP1_Loads,Landing_Page_Events,Final_Media_Cost,Account_Opens,Chicago,Washington DC,Denver,Hartford-New Haven,Minneapolis,San Diego,Philadelphia,San Francisco
107,255866.0,531,2,357,1646.580928,24,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
108,388607.0,774,4,434,2537.699654,63,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
109,329378.0,831,6,355,2116.389121,64,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
110,418910.0,845,1,499,2418.099370,117,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
111,315385.0,594,2,372,1869.038813,338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [81]:
train_dataset = df_final.sample(frac=0.8, random_state=0)
test_dataset = df_final.drop(train_dataset.index)

In [82]:
train_stats = train_dataset.describe()
train_stats.pop('Account_Opens')
train_stats = train_stats.transpose()
train_stats

count     90.000000
mean      95.511111
std       71.784516
min       11.000000
25%       37.500000
50%       65.500000
75%      137.750000
max      338.000000
Name: Account_Opens, dtype: float64

,count,mean,std,min,25%,50%,75%,max
Impressions,90.0,476355.666667,168282.502686,126197.000000,349196.500000,486527.500000,611755.00000,968441.000000
Clicks,90.0,725.400000,272.536599,208.000000,537.750000,700.000000,929.25000,1553.000000
AP1_Loads,90.0,3.522222,2.853060,0.000000,2.000000,3.000000,5.00000,18.000000
Landing_Page_Events,90.0,401.166667,149.166919,93.000000,296.250000,417.000000,505.75000,744.000000
Final_Media_Cost,90.0,5130.175568,2603.325070,577.006957,3168.161492,4732.346758,7089.59627,11693.965047
Chicago,90.0,0.133333,0.341839,0.000000,0.000000,0.000000,0.00000,1.000000
Washington DC,90.0,0.122222,0.329377,0.000000,0.000000,0.000000,0.00000,1.000000
Denver,90.0,0.133333,0.341839,0.000000,0.000000,0.000000,0.00000,1.000000
Hartford-New Haven,90.0,0.122222,0.329377,0.000000,0.000000,0.000000,0.00000,1.000000
Minneapolis,90.0,0.111111,0.316030,0.000000,0.000000,0.000000,0.00000,1.000000


In [83]:
train_labels = train_dataset.pop('Account_Opens')
test_labels = test_dataset.pop('Account_Opens')

In [84]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [85]:
def build_model():
    model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
  ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
    return model

In [86]:
model = build_model()

In [ ]:
feature_col = [Impressions, Clicks, Ap1_Loads, Landing_Page_Events, Final_Media_Cost]

In [38]:
datetime.date(2016,1,1)

datetime.date(2016, 1, 1)

In [56]:
df_ao = df_ao[(df_ao['vis_date'] >= pd.Timestamp(2019,4,1)) & (df_ao['channel_id'] == 'Direct')]

In [47]:
df_dcm = df_dcm[df_dcm['Date'] >= pd.Timestamp(2019,4,1)]

In [66]:
df_dcm_imp = df_dcm.groupby(['Date', 'Market'])[['Impressions']].sum().reset_index()

In [64]:
df_ao_chicago = df_ao[df_ao['Market'] == 'Chicago']

In [63]:
df_ao['Market'] = df_ao['Market'].str.strip()

In [65]:
df_ao_chicago

,vis_date,dma,channel_id,e119_count,Market,Month,Year,Previous Month,Next Year Date,WeekNum,Pod,Test.Control
36019,2019-04-01,Chicago (602),Direct,28,Chicago,April,2019,March,2020-03-31,14,1,Test
36101,2019-04-02,Chicago (602),Direct,30,Chicago,April,2019,March,2020-04-01,14,1,Test
36185,2019-04-03,Chicago (602),Direct,26,Chicago,April,2019,March,2020-04-02,14,1,Test
36268,2019-04-04,Chicago (602),Direct,22,Chicago,April,2019,March,2020-04-03,14,1,Test
36351,2019-04-05,Chicago (602),Direct,13,Chicago,April,2019,March,2020-04-04,14,1,Test
36431,2019-04-06,Chicago (602),Direct,14,Chicago,April,2019,March,2020-04-05,14,1,Test
36509,2019-04-07,Chicago (602),Direct,24,Chicago,April,2019,March,2020-04-06,14,1,Test
36586,2019-04-08,Chicago (602),Direct,29,Chicago,April,2019,March,2020-04-07,15,1,Test
36666,2019-04-09,Chicago (602),Direct,23,Chicago,April,2019,March,2020-04-08,15,1,Test
36745,2019-04-10,Chicago (602),Direct,25,Chicago,April,2019,March,2020-04-09,15,1,Test


In [72]:
df1 = df_dcm_imp[(df_dcm_imp['Market'] == 'Chicago') & (df_dcm_imp['Date'] <= pd.Timestamp(2019,7,6))]

In [74]:
df2 = df_ao_chicago[['vis_date', 'e119_count']]

In [75]:
df1.head()

,Date,Market,Impressions
1,2019-04-01,Chicago,54716.0
9,2019-04-02,Chicago,60847.0
17,2019-04-03,Chicago,68985.0
25,2019-04-04,Chicago,72897.0
33,2019-04-05,Chicago,74045.0


In [76]:
df2.head()

,vis_date,e119_count
36019,2019-04-01,28
36101,2019-04-02,30
36185,2019-04-03,26
36268,2019-04-04,22
36351,2019-04-05,13


In [78]:
np.corrcoef(df1['Impressions'], df2['e119_count'])

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [79]:
df1.shape

(97, 3)

In [80]:
df2.shape

(96, 2)

In [84]:
df1.to_csv('df1.csv')

In [85]:
df2.to_csv('df2.csv')

In [83]:
pd.merge(df1, df2, how='left', on=['Date'])

KeyError: 'Date'